# 1.验证模型结构

In [ ]:
import torch
import numpy as np
from base_module import *

from pymahjong import *
from pymahjong import MahjongPyWrapper as pm
from pymahjong.myEnv_pymahjong import myMahjongEnv

env = myMahjongEnv()
env.reset()


# 检测GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 测试用输入
batch =  []
batch_size = 16
feature_channels = 35
feature_height = 4
feature_width = 9

action_space = 47

while len(batch) < batch_size:
    while not env.is_over():
        curr_pid = env.get_curr_player_id()
        valid_actions = env.get_valid_actions()
        action = np.random.choice(valid_actions)
        env.step(player_id=curr_pid, action=action)
    print(env.get_payoffs())
    for i in range(4):
        batch.append(env.get_observation_with_return(i))
    env.reset()
collator = myCollator()

In [ ]:
from base_module import *
import torch
import torch.nn as nn
import numpy as np
from transformers import GPT2Model, GPT2Config


# 检测GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)
# 创建模型
config = GPT2Config(n_embd=512, n_layer=8, n_head=8, n_positions=128)
model = Policy_Network(config).to(device)
data = collator(batch)
model.eval()


In [ ]:

output = model.forward(**data)
print("action_logits:", output["action_logits"].shape)
print(output["action_logits"][0])
print("probs:", output["action_probs"][0])
# print("loss:", output["loss"])
print("action:", output["action"])


# 2.验证麻将环境

In [1]:
from pymahjong import *
from pymahjong import MahjongPyWrapper as pm
from pymahjong.myEnv_pymahjong import myMahjongEnv

env = myMahjongEnv()
env.reset()

In [ ]:
print(env.get_valid_actions())
print(env.legal_actions_mask_record)

In [ ]:
while not env.is_over():
    curr_pid = env.get_curr_player_id()
    valid_actions = env.get_valid_actions()
    action = np.random.choice(valid_actions)
    env.step(player_id=curr_pid, action=action)
print(env.get_payoffs())

#### 展示当前phase和合法动作列表

In [ ]:
phase, aviable_action = env._proceed()
print("phase:", phase)
for idx, action in enumerate(aviable_action):
    print(idx, action.to_string())

valid_action = env.get_valid_actions()
print("valid_action:", valid_action)

#### 执行动作

In [4]:
env.step(2, 0)

In [ ]:
for i in range(4):
    print("player", i, "hand:", env._get_hand_tiles(i))

In [ ]:
obs = env.get_observation(1)
print("tiles_features shape:", obs['tiles_features'].shape)
print("oya shape:", obs['info']['oya'].shape)
print("riichi_sticks shape:", obs['info']['riichi_sticks'].shape)
print("action_list shape:", obs['action_list'].shape)
print("action_list:", obs['action_list'])
print("self_action_mask shape:", obs['self_action_mask'].shape)
print("self_action_mask:", obs['self_action_mask'])
print("sum self_action_mask:", obs['self_action_mask'].sum())
print("attention_mask shape:", obs['attention_mask'].shape)
print("attention_mask:", obs['attention_mask'])
print("Q shape:", obs['Q_values'].shape)
print("Q:", obs['Q_values'])
print("legal_action_mask shape:", obs['legal_action_mask'].shape)
print("legal_action_mask:", obs['legal_action_mask'])

In [ ]:
for i in range(4):
    print("player", i, "fuuros:", env._get_fuuros(i))
    print("points:", env._get_points(i))

In [ ]:
obs['info']

# 3.使用模型来决策

In [1]:
import torch
import numpy as np
from base_module import *

from pymahjong import *
from pymahjong import MahjongPyWrapper as pm
from pymahjong.myEnv_pymahjong import myMahjongEnv

env = myMahjongEnv()
env.reset()


# 检测GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# for i in range(10):
#     curr_pid = env.get_curr_player_id()
#     valid_actions = env.get_valid_actions()
#     action = np.random.choice(valid_actions)
#     env.step(player_id=curr_pid, action=action)

config = GPT2Config(n_embd=512, n_layer=8, n_head=8, n_positions=128)
model = Policy_Network(config).to(device)
model.eval()

collator = myCollator()
for i in range(100):
    while not env.is_over():
        curr_pid = env.get_curr_player_id()
        obs = env.get_observation(curr_pid)
        input = {
        "tiles_features": torch.tensor(obs['tiles_features'], dtype=torch.float32).to(device),
        "oya": torch.tensor(obs['info']['oya'], dtype=torch.float32).unsqueeze(0).to(device),
        "riichi_sticks": torch.tensor(obs['info']['riichi_sticks'],dtype=torch.float32).unsqueeze(0).to(device),
        "action_list": torch.tensor(obs['action_list'],dtype=torch.long).unsqueeze(0).to(device),
        "attention_mask": torch.tensor(obs['attention_mask'],dtype=torch.long).unsqueeze(0).to(device),
        "legal_action_mask": torch.tensor(obs['legal_action_mask'], dtype=bool).to(device)
        }
        output = model.inference(**input)
        action = output["action"].item()
        env.step(player_id=curr_pid, action=action)
        # print("player:", curr_pid, "action:", action)
    print(env.get_payoffs())
    env.reset()



# print("action:", output["action"])

device: cuda
player: 0 action: 31
player: 1 action: 31
player: 2 action: 11
player: 3 action: 31
player: 0 action: 9
player: 1 action: 5
player: 2 action: 4
player: 3 action: 2
player: 0 action: 23
player: 1 action: 36
player: 1 action: 28
player: 2 action: 2
player: 3 action: 36
player: 3 action: 4
player: 0 action: 23
player: 1 action: 34
player: 1 action: 11
player: 2 action: 35
player: 2 action: 14
player: 3 action: 36
player: 3 action: 16
player: 0 action: 37
player: 0 action: 9
player: 1 action: 2
player: 2 action: 15
player: 3 action: 18
player: 0 action: 11
player: 1 action: 16
player: 2 action: 33
player: 3 action: 30
player: 0 action: 28
player: 1 action: 26
player: 0 action: 37
player: 2 action: 36
player: 0 action: 29
player: 1 action: 5
player: 2 action: 18
player: 3 action: 6
player: 0 action: 13
player: 1 action: 15
player: 2 action: 10
player: 3 action: 38
player: 3 action: 28
player: 0 action: 31
player: 1 action: 33
player: 2 action: 1
player: 3 action: 6
player: 0 ac

In [ ]:
print("action_probs:", output["action_probs"])
print("action:", output["action"].item(), "action_probs:", output["action_probs"][0][output["action"]].item()*100, "%")